## Window Theorem (3)

For two separation-identical aircraft $i$ and $j$ where $r_i \le r_j$ and $lt_i \le lt_j$, a *complete-order* can be inferred such that $i$ should precede $j$. Since an aircraft can always be delayed to meet the start of its hard time window, and $t_x \le t'_x$ for all aircraft in corresponding positions of sequences $s$ and $s'$, any time-window violation is no worse in $s$ than in $s'$. Hence, sequencing $i$ before $j$ cannot increase the number or severity of hard time-window violations. Firstly, we import (relevant) libraries, and initialise a `Solver` instance.


In [22]:
from z3 import *
from itertools import product
from functools import reduce

s = Solver()


Then, define some variables

In [23]:

# Introduce aircraft sequence {p1, i, p2, j, p3}
p1, i, p2, j, p3 = Ints('p1 i p2 j p3')
aircraft = [p1, i, p2, j, p3]

# Define some mappings for R (release time) LT (end of time window) and δ (minimum separation)
R  = {ac: Real(f"R_{ac}")  for ac in aircraft}
LT = {ac: Real(f"LT_{ac}") for ac in aircraft}
δ   = {(x, y): Real(f"δ_{x}_{y}") for x, y in product(aircraft, aircraft)}


# Constraints


In [24]:

# Each aircraft ID should be unique ∀(x, y) ∈ S × S, x ≠ y
s.add(Distinct(*aircraft))



# R & LT are non-negative real numbers.
for x in aircraft:
    s.add(R[x] > 0, LT[x] > 0)



# δ maps into a non-negative codomain.
for (x, y) in product(aircraft, aircraft):
    s.add(δ[x, y] >= 0)
    if x == y:
        s.add(δ[x, y] == 0)



# δ-identical i/j
for x in aircraft:
    s.add(δ[i, x] == δ[j, x])
    s.add(δ[x, i] == δ[x, j])


# Symbolic Max


In [25]:

# helper (symbolic max)
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs): return reduce(lambda a, b: zmax(a, b), xs)

def compute_T(seq):
    T = {seq[0]: R[seq[0]]}
    for k in range(1, len(seq)):
        preds = seq[:k]
        T[seq[k]] = zmax(R[seq[k]],
                         zmax_list([T[p] + δ[p, seq[k]] for p in preds]))
    return T



# Window Violations

In [26]:

def window_violations(T):
    return Sum([If(T[a] > LT[a], 1, 0) for a in aircraft])


# Encode Preconditions

In [27]:

# encode s and s'
S       = [p1, i, p2, j, p3]
S_prime = [p1, j, p2, i, p3]

T       = compute_T(S)
T_prime = compute_T(S_prime)

WV      = window_violations(T)
WV_p    = window_violations(T_prime)



# theorem assumptions
s.add(R[i] < R[j])
s.add(LT[i] < LT[j])



# S' Is feasible
for ac in aircraft:
    s.add(T_prime[ac] < LT[ac])



# hypothesis
s.add(WV > WV_p)


# Check Solver

In [28]:

if s.check() == z3.sat:
    model = s.model()
    print("\nTheorem holds SAT (counterexample found).\nModel:")
    for d in model.decls():
        print(f" - {d.name()} = {model[d]}")
elif s.check() == z3.unsat:
    print("\nUNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance if s' is feasible.")



UNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance if s' is feasible.
